@author: Md Siamul Islam
# Parking Spot Vacancy/Occupancy recognition deep NN solution-VGG16

In [0]:
# pre-req (if required)

# keras- high-level NN API- CPU&GPU compatibility - backend engine supports: Theano, Tensorflow by default, or CNTK
# saving keras models to disk: use HDF5 or h5py
# can use Theano for Linus systems - running for first time creates JSON but if reconfiguration necessary: create the file and configure backend
# $ pwd
# $ mkdir .keras
# $ cd .keras/
# /.keras$ touch keras.JSON/.keras$ vim keras.json
# go to INSERT mode and enter
# {
# "image_data_format":"channels_first",
# "epsilon": le-07,
# "floatx": "float32",
# "backend": "theano"
# }
# ~
# ~
# //use "image_dim_ordering": "th", for first parameter for older keras version or "tf" for tensorflow



Getting Reproducable results -eliminate randomness - can be good for demo purposes
- setting Random Seed- before building and training the model- random vars  generated same w/ fixed  random seed

In [0]:
# import random as rn
# import numpy as np
# import tensorflow as tf

# import os
# os.environ['PYTHONHASHSEED'] = '0'

# # Set seed for random numbers generated by numpy
# np.random.seed(43)

# # set seed for python random numbers
# rn.seed(1254)

# # set seed for Tensorflow rand number
# tf.set_random_seed(78)


# from keras import backend as k

# # forcing tensorflow to use single thread
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# k.set_session(sess)

# all keras codes follow this code block

# VGG16 model ver 2
# fine tuned with Keras

In [2]:
import tensorflow as tf
import keras
from keras import backend as k
#from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.layers.core import Dense, Flatten
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Activation
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
#from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import layers



vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()  # can work on 1000 different categories but need only 2 - need to modify the model for our purpose

# .model -transform into Sequential model(object)
type(vgg16_model)

model = Sequential() # model set to sequential object -currently model has no layers
# iterating over all layers within vgg16 model and adding those layers to the sequential model
# now we have a model of sequential type and don't need to work with functional API
# for layer in vgg16_model.layers:
#     model.add(layer)

# alternatively, iterating over all except last layer in vgg16, adding to seq model- no pop() required
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

model.summary()

# model classifies images- 1 of 1000 categories, not what we want- need to take off the last predictions output dense layer- use pop()
# model.layers.pop()
# model.summary()


# no longer working on VGG16 model variable but the new model variable which is a sequential model
# setting the layers shown in summary to false to freeze/exclude in future training so the weights are not updated
# good for fine  tuning a model -don't need to update layer weights and keep them same
for layer in model.layers:
    layer.trainable = False

# instead of the last prediction layer that we popped off,
# a new dense layer is added to the model that classifies images to two categories -for 'empty' and 'occupied'
model.add(Dense(2, activation='softmax'))
model.summary()




Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Preprocessing Data for training using keras

# Image Preparation
-for VGG16 image clasifier

In [0]:
# organize directory structure - place image on disk (unless using libraries to handle this)
# eg: directories: test, train, valid in some directory spot
# eg: spot/train/ has directories: empty and occupied containing images corresponding to those labels
# eg valid directory has empty and occupied directories with images for validation - set validation % parameter- between 0 and 1- to take from dataset
# typically dump all sorts of unorganized labelled images into test directory; but for known labelled images to test, may create empty and occupied folders w/images to test on


In [0]:
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

# train_labels = [] # corresponding label for sample - needs to be in numpy array
# train_samples = [] # actual image data - numpy array or list of numpy array for eg num data purpose

train_path = 'content/sample_data/train'
test_path = 'content/sample_data/test'
valid_path = 'content/sample_data/valid'

# ImageDataGenerator -keras object- to get it formatted to be used by keras model
# creates batches of normalized data
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['empty', 'occupied'], batch_size=28)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['empty', 'occupied'], batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['empty', 'occupied'], batch_size=10)

# Plotting image with labels on Jupyter Notebook- from git
def plots(ims, figsize(12,6), rows=1, interp=False, title=None):
  if type(ims[0]) is np.ndarray:
    ims = np.array(ims).astype(np.uint8)
    if(ims.shape[-1] != 3):
      ims = ims.transpose((0,2,3,1))
      f = plt.figure(figsize=figsize)
      cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
      for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
          sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')


imgs, labels = next(train_batches) # gets another batch of size 10 from total of images in training set - gets new batch every time its plotted/run
plots(imgs, titles=labels)        # observe labels and see if correct and uniform- encoding - eg: [1. 0.] and [0. 1.] for empty and occupied respectively or vice versa
# or might use Binary classification instead of categorical to get 1 or 0 i.e 1-D label instead of 2D vector format above 

# To view which label belongs to which class as assigned by VGG:
test_batches.class_indices   # call on the image data generators -returns the dictionary containing mapping from class names to class indices
# returned value corresponds the ....................



# include code block for grayscale conversion, img normalization-eg.size, etc. & other pre-processing as desired


# Data Augmentation (optional)- modify existing data to create new data for training- if dataset is small/larger, varied dataset benefitial -may use Keras
# -eg. flip horizontally/vertically, rotate, zoom in/out, vary color, crop 
# may reduce overfitting

# insert code-block


---------------------------------------------------------------------------------------------------------------------------------------------

# Train the new vgg16 model


In [0]:
# compile the new model using Adam optimization function with learning rate 0.0001 and provided functions and metrics
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# passing in images from image data generator(code block missing) in training set, steps_per_epoch= (images in dataset)/(batch size), valid set generated from image generator(missing code block),
# ..., verbose set to level 2
model.fit_generator(train_batches, steps_per_epoch=7,
                    validation_data=valid_batches, validation_steps=4, epochs=9, verbose =2)

# observe metrics from training- compare with 3X64 node CNN created previously- use the better one
# also check for overfitting

# Predictions

In [0]:
#
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

# 0th index so that empty/occupied gets values 0/1 or vice versa
test_labels = test_labels[:,0]

# set steps according to : test set image count (eg. 10) and batch size: (eg. 10), so it takes 1 step to run through the batch of imgs
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

# create confusion matrix variable
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

cm_plot_labels = ['empty', 'occupied']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix') # plots predicted labels vs true labels on x-y axis -observe values
# accuracy might depend on amount of data used for training, epochs (can use library/tool "..." to get optimal value), and other tweaks
# compare confusion matrix with previously built 3X64 CNN model - use the best one


# Confusion Matrix
-Plot Predicted labels vs True labels 

In [0]:
# insert code (included above)
#.
#.
#.
# cm_plot_labels = ['empty', 'occupied']
# plot_confusion_matrix(cm, cm_plot_labels, title= 'Confusion Matrix')


# Saving the VGG16 Model

In [0]:
model.save('SpotNN_vgg16.h5') # will save to directory where the code is executed by default unless path is specified


# Loading saved model

In [0]:
# from keras.models import load_model

# new_model = load_model('SpotNN_vgg16.h5')

# new_model.summary() # will yeild same architecture that allows for recreating the model, the weights, training configuration i.e. loss, optimizer, state of optimizer that allows resuming training from exactly where it was left off
# new_model.get_weights()



# To save only the architecture of the Model: json
-weights and training configuration excluded

and model reconstructions from JSON

In [0]:
# # saving as JSON
# json_string = model.to_json()

# # saving as YAML
# #yaml_string = model.to_yaml

# json_string

# # reconstruction of model from JSON
# from keras.models import model_from_json

# model_architecture = model_from_json(json_string)

# #model reconstruction from YAML
# # from keras.models import model_from_yaml
# # model = model_from_yaml(yaml_string)

# model_architecture.summary()


# To save only Weights of the model


In [0]:
# model.save_weights('vgg16_model_weights.h5')
# model2 = Sequential([
#                      Dense(16, input_shape=(1,), activation='relu'),
#                      Dense(32, activation='relu'),
#                      Dense(2, activation='softmax')
# ])

# model2.load_weights('vgg16_model_weights.h5')


# Deploy Model to Web Service- Using Flask --continue to next- ignore

Ignore next code block --continue from predict_app.py

In [2]:
# sample_app.py
# very basic info below for explanation/docu purposes(skip to uncommented code segment)
# Web app makes an http/? call/request to a Flask web service (backend written in Python) hosting the VGG16 model
# -requests model a prediction for the provided image- response received by the webpage/app (can be written in any language)
# Flask- microFramework/minimal web-application framework in Python

# install Flask
# !pip install Flask
# conda install -c anaconda flask

# webservice- create python file under eg. "home_dir"/flask_apps/sample_app.py

# from flask import Flask
# app = Flask(__name__)  #creates instance of flask class, __name__ name of applications module- used for single module

# @app.route('/sample') # sample set as endpoint
# followed by fn() when user accesses/browses to the specified '/sample' or '"ip-address"/sample' endpoint -outputs 'flask is running'
# def running():
#   return 'Flask is running!'

# enter terminal and type in:
#~/flask_apps$ export FLASK_APP=sample_app.py
# here working directory is flask_apps/.   cd to directory or provide full path if not in dir

#~/flask_apps$ flask run --host=0.0.0.0    # starts flask; host parameter specified makes the web service visible and publicly available to all computers on the network- if not specified: access from local machine only

# default port: 5000
# http://localhost:5000/sample  # sample is endpoint created for service - access from local machine - else, get ip address on machine running Flask -for remote machine to browse to: eg: 10.0.0.5:5000/sample
# displays " Flask is running"  - log in information displayed in terminal
# pass or request data to flask from Front End and receive a response from the back end web service flask

# Sending and Receiving Data using Flask

# host web service & web app -Flask


# set up directory structure
# eg: under flask_apps, create a "static" directory to store static files ex: html that flask serves the webpage for webapp. Put python code under flask_apps.
# 

# from flask import request
# from flask import jsonify
# from flask import Flask

# app = Flask(__name__)

# To send data to webserver, use eg: 
# @app.route('/greet',methods=['POST']) 
# #methods parameter: what kinds of http request are allowed for this endpoint - 'POST' : indicates send data to the server along with the request: sends name to flask webservice- name entered= data contained in http post request sent from webapp to 'greet' endpoint


# def greet():  # todo when post request received from endpoint
      # message = request.get_json(force=True) # request.: to access data sent to endpoint; get_json on request: gives message from client in JSON- key,value pairs 
      # name = message['name']
      # response = {   # sends back to webapp
      #     'greeting': 'Howdy, ' + name + '!'   # dictionary...
      # }
      # return jsonify(response)   # converts python dictionary to json -> json to webapp



# cd to flask_apps dir
# export FLASK_APP=hello_app.py      
# flask run --host=0.0.0.0

#backend now running
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# from flask import request
# from flask import jsonify
# from flask import Flask



# Deploy Model to Web Service- Using Flask
# predict_app.py
### Modify as required for FrontEnd/BackEnd spec 
### Follow instructions to execute successfully -in.prog

In [0]:
# predict_app.py
# Build backend Flask application to host VGG16 model for predictions
# under flask_apps directory, create predict_app.py file containing code for web service. Place vgg16 model in form of .h5 file within this dir.
# may organize front end files here in appropriate directories and specify/mod paths in code
# Modify as required for FrontEnd/BackEnd spec


#code for predict app
import numpy as np
import base64
from flask import jsonify
from flask import Flask
from flask import request
import io
from PIL import Image
import keras
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Sequential


app = Flask(__name__)  # instance of flask class created

def get_model():
  global model
  model = load_model('SpotNN_vgg16.h5')    # load the saved vgg16 model into memory
  print(" * Model is Loaded!")


def preprocess_image(image, target_size):
  if image.mode != "RGB":
      image = image.convert("RGB")
  image = image.resize(target_size)
  image = img_to_array(image)   # to numpy array
  image= np.expand_dims(image, axis=0)  # expands the dimensions of img

  return image    # preprocessed image returned to be passed to keras model


print(" * Loading VGG16 Keras model...")
get_model()  # model loaded into memory - doesn't require reloading every time a request comes into endpoint


@app.route("/predict", methods=["POST"])   # Endpoint: "/predict"  -image data sent to endpoint to GET prediction
def predict():
    message = request.get_json(force=True)
    encoded = message['image']  # encoded assigned value associated with key called image json data stored in message variable -atleast one (key,value) pair where key = image, value associated with key is base64 encoded image sent by client
    decoded = base64.b64decode(encoded) # now decoded assigned decoded img data
    image = Image.open(io.BytesIO(decoded))  # image object with the decoded message
    processed_image = preprocess_image(image, target_size=(224,224))   # preprocess then pass it to model for prediction

    prediction = model.predict(processed_image).tolist()   # returns numpy array with predictions

# send this prediction as JSON back to the client
    response = {
        'prediction':{     # dictionary with key called prediction
            'empty': prediction[0][0],  # empty key set to 0th element of 0th list of prediction list
            'occupied': prediction[0][1]    # occupied key set to 1st element of 0th list of prediction list
        }    # only 1 image predicted, so prediction list contains only 1 embedded list with probability for empty and probability for occupied
    }
    return jsonify(response)    # converts python dictionary to JSON--return JSON to FrontEnd


# type into powershell/.../...

# ~/flask_apps$ export FLASK_APP=predict_app.py
# ~/flask_apps$ flask run --host=0.0.0.0

# output:
# Using XXXX backend.
# * Loading Keras model...
# * Model loaded!
# * Serving Flask app "predict_app"
# * Running on http://0.0.0.0:5000/ (press CTRL+C to quit)


# For Powershell in windows machines: 
# Powershell: Access image >>Format image data >>Make HTTP POST request >>Get Response for img
# PS C:\imgs> $fileName = 'C:\pics\empty3.PNG'
# PS C:\imgs> $bytes = [IO.File]::ReadyAllBytes($fileName)
# PS C:\imgs> $base64Image = [Convert]::ToBase64String($bytes)
# PS C:\imgs> $message = @{ image = $base64Image }
# PS C:\imgs> $jsonified = ConvertTo-Json $message
# PS C:\imgs> $response = Invoke-RestMethod -Method Post -Url "http"//??.?.?.?:5000/predict" -Body $jsonified
# PS C:\imgs> $response.prediction | format-list

# sample output:

# empty:    X.XXXXXXXXXXXE-XX
# occupied: X.XXXXXXXXXXXXXXX

# or Curl to use with Bash terminal:
# ..MINGW64 /c/imgs
# $ fileName=C\:/imgs/empty3.PNG
# ..MINGW64 /c/imgs
# $ base64Image=$(base64 $fileName)
# ..MINGW64 /c/imgs
# $ jsonified="{\"image\":\"${base64Image}\"}"
# ..MINGW64 /c/imgs
# $ echo $jsonified >> data.json
# ..MINGW64 /c/imgs
# $curl -X POST --data @data.json http://??.?.?.?:5000/predict

# sample output:
# .
# .
# .
# .


# Make the Front-End Web App to call the predict endpoint